In [ ]:
import copy
import datetime as dt
from datetime import datetime
import importlib  # needed so that we can reload packages
import logging
import os
import pathlib
import sys
import time
import warnings
from typing import Union, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from utils.logger_utils import setup_clean_logger, mute_external_loggers

# SISEPUEDE imports
from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf
import sisepuede.core.attribute_table as att
import sisepuede.manager.sisepuede_examples as sxl
import sisepuede.manager.sisepuede_file_structure as sfs
import sisepuede.manager.sisepuede_models as sm
import sisepuede.visualization.plots as svp

# --- Runtime configuration ---
warnings.filterwarnings("ignore")

# Set up a clean logger for your notebook
logger = setup_clean_logger("notebook", logging.INFO)
logger.info("Notebook started successfully.")

# Mute logs from sisepuede to avoid duplication
mute_external_loggers(["sisepuede"])


In [ ]:
%load_ext autoreload
%autoreload 2

### Initial Set up

Make sure to edit the config yaml under ssp_modeling/config_files/config.yaml

You can also create a new config yaml



In [ ]:
# Set up dir paths

CURR_DIR_PATH = pathlib.Path(os.getcwd())
SSP_MODELING_DIR_PATH = CURR_DIR_PATH.parent
PROJECT_DIR_PATH = SSP_MODELING_DIR_PATH.parent
DATA_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("input_data")
RUN_OUTPUT_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("ssp_run_output")
SCENARIO_MAPPING_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("scenario_mapping")
CONFIG_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("config_files")
TRANSFORMATIONS_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("transformations")
MISC_DIR_PATH = SSP_MODELING_DIR_PATH.joinpath("misc")
STRATEGIES_DEFINITIONS_FILE_PATH = TRANSFORMATIONS_DIR_PATH.joinpath("strategy_definitions.csv")
STRATEGY_MAPPING_FILE_PATH = MISC_DIR_PATH.joinpath("strategy_mapping.yaml")

In [ ]:
from ssp_transformations_handler.GeneralUtils import GeneralUtils
from ssp_transformations_handler.TransformationUtils import TransformationYamlProcessor, StrategyCSVHandler

# Initialize general utilities
g_utils = GeneralUtils()

In [ ]:
# Load config file, double check your parameters are correct

YAML_FILE_PATH = os.path.join(CONFIG_DIR_PATH, "config.yaml")
config_params = g_utils.read_yaml(YAML_FILE_PATH)

country_name = config_params['country_name']
ssp_input_file_name = config_params['ssp_input_file_name']
ssp_transformation_cw = config_params['ssp_transformation_cw']
energy_model_flag = config_params['energy_model_flag']
set_lndu_reallocation_factor_to_zero_flag = config_params['set_lndu_reallocation_factor_to_zero']

# Print config parameters
logger.info(f"Country name: {country_name}")
logger.info(f"SSP input file name: {ssp_input_file_name}")
logger.info(f"SSP transformation CW: {ssp_transformation_cw}")
logger.info(f"Energy model flag: {energy_model_flag}")
logger.info(f"Set lndu reallocation factor to zero flag: {set_lndu_reallocation_factor_to_zero_flag}")

In [ ]:
def get_file_structure(
    y0: int = 2015,
    y1: int = 2060,
) -> Tuple[sfs.SISEPUEDEFileStructure, att.AttributeTable]:
    """Get the SISEPUEDE File Structure and update the attribute table
        with new years.
    """
    # setup some SISEPUEDE variables and update time period
    file_struct = sfs.SISEPUEDEFileStructure(
        initialize_directories = False,
    )
 
    # get some keys
    key_time_period = file_struct.model_attributes.dim_time_period
    key_year = file_struct.model_attributes.field_dim_year
 
 
    ##  BUILD THE ATTRIBUTE AND UPDATE
 
    # setup the new attribute table
    years = np.arange(y0, y1 + 1, ).astype(int)
    attribute_time_period = att.AttributeTable(
        pd.DataFrame(
            {
                key_time_period: range(len(years)),
                key_year: years,
            }
        ),
        key_time_period,
    )
 
    # finally, update the ModelAttributes inside the file structure
    (
        file_struct
        .model_attributes
        .update_dimensional_attribute_table(
            attribute_time_period,
        )
    )
 
    # return the tuple
    out = (file_struct, attribute_time_period, )
 
    return out
 
# # setup models
# models = sm.SISEPUEDEModels(
#     matt,
#     allow_electricity_run = True,
#     fp_julia = _FILE_STRUCTURE.dir_jl,
#     fp_nemomod_reference_files = _FILE_STRUCTURE.dir_ref_nemo,
#     initialize_julia = True, 
# )

In [ ]:
# Set up SSP objects
INPUT_FILE_PATH = DATA_DIR_PATH.joinpath(ssp_input_file_name)

# model attributes and associated support classes
_EXAMPLES = sxl.SISEPUEDEExamples()
_FILE_STRUCTURE, _ATTRIBUTE_TABLE_TIME_PERIOD = get_file_structure()
matt = _FILE_STRUCTURE.model_attributes
regions = sc.Regions(matt, )
time_periods = sc.TimePeriods(matt, )

### Making sure our input file has the correct format and correct columns
We use an example df with the complete fields and correct format to make sure our file is in the right shape

In [ ]:
INPUT_FILE_PATH

In [ ]:
##  BUILD BASE INPUTS
df_inputs_raw = pd.read_csv(INPUT_FILE_PATH)

# pull example data to fill in gaps
df_example_input = _EXAMPLES("input_data_frame")

In [ ]:
# Double checking that our df is in the correct shape (Empty sets should be printed to make sure everything is Ok!)
g_utils.compare_dfs(df_example_input, df_inputs_raw)

In [ ]:
# Ensure if time_period field exist
if 'time_period' not in df_inputs_raw.columns:
    logger.info("Adding 'time_period' column to df_inputs_raw")
    df_inputs_raw = df_inputs_raw.rename(columns={'period':'time_period'})
else:
    logger.info("'time_period' column already exists in df_inputs_raw")

In [ ]:
# Fixes differences and makes sure that our df is in the correct format.
# Note: Edit this if you need more changes in your df

df_inputs_raw_complete = g_utils.add_missing_cols(df_example_input, df_inputs_raw.copy())
df_inputs_raw_complete.head()

In [ ]:
# Double checking that our df is in the correct shape (Empty sets should be printed to make sure everything is Ok!)
g_utils.compare_dfs(df_example_input, df_inputs_raw_complete)

In [ ]:
# check region field
df_inputs_raw_complete['region'].unique()

In [ ]:
# Set region to country name
df_inputs_raw_complete['region'] = country_name
df_inputs_raw_complete['region'].head()

In [ ]:
crop_yield_factor = matt.get_variable("Crop Yield Factor")
crop_yield_factor.get_from_dataframe(df_inputs_raw_complete).iloc[0]

In [ ]:
df_inputs_raw_complete['yf_agrc_cereals_tonne_ha'] *= 0.74

In [ ]:
df_inputs_raw_complete['yf_agrc_sugar_cane_tonne_ha'] *= 0.85

In [ ]:
initial_cropland_area_proportion = matt.get_variable("Initial Cropland Area Proportion")
initial_cropland_area_proportion.get_from_dataframe(df_inputs_raw_complete).iloc[0]

In [ ]:
crop_demand_income_elasticity = matt.get_variable("Crop Demand Income Elasticity")
crop_demand_income_elasticity.get_from_dataframe(df_inputs_raw_complete).iloc[0]

In [ ]:
initial_cropland_area_proportion = matt.get_variable("Initial Land Use Area Proportion")
initial_cropland_area_proportion.get_from_dataframe(df_inputs_raw_complete).iloc[0]

In [ ]:
# List of land use fraction columns in the SISEPUEDE input
landuse_cols = [
    "frac_lndu_initial_croplands",
    "frac_lndu_initial_flooded",
    "frac_lndu_initial_forests_mangroves",
    "frac_lndu_initial_forests_primary",
    "frac_lndu_initial_forests_secondary",
    "frac_lndu_initial_grasslands",
    "frac_lndu_initial_other",
    "frac_lndu_initial_pastures",
    "frac_lndu_initial_settlements",
    "frac_lndu_initial_shrublands",
    "frac_lndu_initial_wetlands"
]

df_inputs_raw_complete.loc[:, "frac_lndu_initial_forests_primary"] = 0.11   # ~11%
df_inputs_raw_complete.loc[:, "frac_lndu_initial_settlements"]    = 0.013  # ~1.3%
df_inputs_raw_complete.loc[:, "frac_lndu_initial_shrublands"]     = 0.29   # ~29%


# Compute the row-wise sum of all land use fractions
row_sum = df_inputs_raw_complete[landuse_cols].sum(axis=1)

# Avoid division by zero just in case (should not happen, but safe)
row_sum = row_sum.replace(0, pd.NA)

# Divide each land use column by the row-wise sum
df_inputs_raw_complete[landuse_cols] = (
    df_inputs_raw_complete[landuse_cols].div(row_sum, axis=0)
)

In [ ]:
initial_cropland_area_proportion.get_from_dataframe(df_inputs_raw_complete).iloc[0]

In [ ]:
import pandas as pd

# Lista de prefijos de productos industriales
product_prefixes = [
    "frac_inen_energy_agriculture_and_livestock_",
    "frac_inen_energy_cement_",
    "frac_inen_energy_chemicals_",
    "frac_inen_energy_electronics_",
    "frac_inen_energy_glass_",
    "frac_inen_energy_lime_and_carbonite_",
    "frac_inen_energy_metals_",
    "frac_inen_energy_mining_",
    "frac_inen_energy_other_product_manufacturing_",
    "frac_inen_energy_paper_",
    "frac_inen_energy_plastic_",
    "frac_inen_energy_recycled_glass_",
    "frac_inen_energy_recycled_metals_",
    "frac_inen_energy_recycled_paper_",
    "frac_inen_energy_recycled_plastic_",
    "frac_inen_energy_recycled_rubber_and_leather_",
    "frac_inen_energy_recycled_textiles_",
    "frac_inen_energy_recycled_wood_",
    "frac_inen_energy_rubber_and_leather_",
    "frac_inen_energy_textiles_",
    "frac_inen_energy_wood_"
]

# Targets para TODOS los productos
targets = {
    "electricity": 0.48,
    "natural_gas": 0.27,
    "oil": 0.17
}

# Iterar sobre cada prefijo / producto
for prefix in product_prefixes:

    # Identificar las columnas de ese producto
    cols = [c for c in df_inputs_raw_complete.columns if c.startswith(prefix)]
    if len(cols) == 0:
        continue  # si no hay columnas, pasar al siguiente producto

    # Columnas específicas que deben fijarse
    target_cols = {
        prefix + "electricity": targets["electricity"],
        prefix + "natural_gas": targets["natural_gas"],
        prefix + "oil": targets["oil"]
    }

    # Aplicar valores fijos a TODAS las filas
    for col, val in target_cols.items():
        if col in df_inputs_raw_complete.columns:
            df_inputs_raw_complete.loc[:, col] = val

    # Reescalar el resto para que sumen 1
    for i in df_inputs_raw_complete.index:

        row = df_inputs_raw_complete.loc[i, cols]

        fixed_sum = sum(val for key, val in target_cols.items() if key in row.index)

        remaining = 1 - fixed_sum

        other_cols = [c for c in cols if c not in target_cols]
        orig_sum_other = row[other_cols].sum()

        if orig_sum_other > 0:
            df_inputs_raw_complete.loc[i, other_cols] = (
                row[other_cols] / orig_sum_other * remaining
            )
        else:
            df_inputs_raw_complete.loc[i, other_cols] = 0

In [ ]:
industrial_energy_fraction_coal = matt.get_variable("Industrial Energy Fuel Fraction Coal")
industrial_energy_fraction_coal.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_coke = matt.get_variable("Industrial Energy Fuel Fraction Coke")
industrial_energy_fraction_coke.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_diesel = matt.get_variable("Industrial Energy Fuel Fraction Diesel")
industrial_energy_fraction_diesel.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_electricity = matt.get_variable("Industrial Energy Fuel Fraction Electricity")
industrial_energy_fraction_electricity.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_electricity.get_from_dataframe(df_inputs_raw_complete).head(35)

In [ ]:
industrial_energy_fraction_electricity.get_from_dataframe(df_inputs_raw_complete).iloc[7].mean()

In [ ]:
industrial_energy_fraction_furnace_gas = matt.get_variable("Industrial Energy Fuel Fraction Furnace Gas")
industrial_energy_fraction_furnace_gas.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_furnace_gasoline = matt.get_variable("Industrial Energy Fuel Fraction Gasoline")
industrial_energy_fraction_furnace_gasoline.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_geothermal = matt.get_variable("Industrial Energy Fuel Fraction Geothermal")
industrial_energy_fraction_geothermal.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_hydrocarbon_gas_liquids = matt.get_variable("Industrial Energy Fuel Fraction Hydrocarbon Gas Liquids")
industrial_energy_fraction_hydrocarbon_gas_liquids.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_hydrogen = matt.get_variable("Industrial Energy Fuel Fraction Hydrogen")
industrial_energy_fraction_hydrogen.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_kerosene = matt.get_variable("Industrial Energy Fuel Fraction Kerosene")
industrial_energy_fraction_kerosene.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_natural_gas = matt.get_variable("Industrial Energy Fuel Fraction Natural Gas")
industrial_energy_fraction_natural_gas.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_natural_gas = matt.get_variable("Industrial Energy Fuel Fraction Natural Gas")
industrial_energy_fraction_natural_gas.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_natural_gas.get_from_dataframe(df_inputs_raw_complete).head(35)

In [ ]:
industrial_energy_fraction_natural_gas.get_from_dataframe(df_inputs_raw_complete).iloc[7].mean()

In [ ]:
industrial_energy_fraction_oil = matt.get_variable("Industrial Energy Fuel Fraction Oil")
industrial_energy_fraction_oil.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
industrial_energy_fraction_oil.get_from_dataframe(df_inputs_raw_complete).head(35)

In [ ]:
industrial_energy_fraction_biomass = matt.get_variable("Industrial Energy Fuel Fraction Solid Biomass")
industrial_energy_fraction_biomass.get_from_dataframe(df_inputs_raw_complete).iloc[7]

In [ ]:
cols = [c for c in df_inputs_raw_complete.columns if c.startswith("yf_agrc_")]
df_inputs_raw_complete[cols].head()

In [ ]:
cols = [c for c in df_inputs_raw_complete.columns if c.startswith("prodinit_ippu_")]
df_inputs_raw_complete[cols].head()

USGS – Mineral Commodity Summary (Cement, 2021) → México ≈ 35 Mt

In [ ]:
df_inputs_raw_complete['prodinit_ippu_cement_tonne'] *= 0.755

More rasonable

In [ ]:
df_inputs_raw_complete['prodinit_ippu_electronics_tonne'] *= 0.02

Low production of glass

In [ ]:
df_inputs_raw_complete['prodinit_ippu_glass_tonne'] *= 2

lime and carbonite

In [ ]:
df_inputs_raw_complete['prodinit_ippu_lime_and_carbonite_tonne'] *= 2

🎯 Producción minera total estimada para México 2020

Sumando solo los grandes rubros:
	•	Caliza: ~180 Mt

	•	Arena + grava: ~130 Mt

	•	Yeso + otros no metálicos: ~15 Mt

	•	Minerales metálicos varios: ~20 Mt
	

In [ ]:
df_inputs_raw_complete['prodinit_ippu_mining_tonne'] *= 1.1

In [ ]:
df_inputs_raw_complete['consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances'] *= 0.3

df_inputs_raw_complete['consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy'] *= 0.3

df_inputs_raw_complete['consumpinit_scoe_gj_per_hh_residential_heat_energy'] *= 0.3

df_inputs_raw_complete['consumpinit_scoe_gj_per_hh_residential_elec_appliances'] *= 0.3



In [ ]:
cols = [c for c in df_inputs_raw_complete.columns if c.startswith("prodinit_ippu_")]
df_inputs_raw_complete[cols].iloc[0]

In [ ]:
df_inputs_raw_complete['elasticity_ippu_rubber_and_leather_production_to_gdp']

In [ ]:
df_inputs_raw_complete['elasticity_ippu_metals_production_to_gdp']

In [ ]:
df_inputs_raw_complete['elasticity_ippu_metals_production_to_gdp'] = 1.0

In [ ]:
df_inputs_raw_complete['elasticity_ippu_rubber_and_leather_production_to_gdp'] = 1.0
df_inputs_raw_complete['elasticity_ippu_rubber_and_leather_production_to_gdp']

In [ ]:
df_inputs_raw_complete['elasticity_ippu_cement_production_to_gdp']

In [ ]:
df_inputs_raw_complete['elasticity_ippu_cement_production_to_gdp'] = 1.0
df_inputs_raw_complete['elasticity_ippu_cement_production_to_gdp']

In [ ]:
DATA_DIR_PATH

In [ ]:
df_inputs_raw_complete.to_csv(DATA_DIR_PATH.joinpath("sisepuede_raw_inputs_MEX_251209.csv"), index=False)

## Let's Modify the  LNDU Reallocation factor

In [ ]:
if set_lndu_reallocation_factor_to_zero_flag:
    df_inputs_raw_complete['lndu_reallocation_factor'] = 0

df_inputs_raw_complete['lndu_reallocation_factor'].mean()

In [ ]:
df_inputs_raw_complete = df_inputs_raw_complete[df_inputs_raw_complete['time_period'].between(0, 35)]
df_inputs_raw_complete['time_period'].unique()

#  Let's try building transformations using this


In [ ]:
transformers = trf.transformers.Transformers(
    {},
    attr_time_period = _ATTRIBUTE_TABLE_TIME_PERIOD,
    df_input = df_inputs_raw_complete,
)

##  Instantiate some transformations. Make sure to run this cell to create the transformations folder for the first time or if you wish to overwrite

In [ ]:
# set an ouput path and instantiate
if not TRANSFORMATIONS_DIR_PATH.exists():
    trf.instantiate_default_strategy_directory(
        transformers,
        TRANSFORMATIONS_DIR_PATH,
    )
else:
    logger.info(f"Directory {TRANSFORMATIONS_DIR_PATH} already exists. Skipping instantiation.")


##  --HERE, CUSTOMIZE YOUR TRANSFORMATIONS AND STRATEGIES--

### Customizing transformations and strategies files using TransformationUtils.py classes

In [ ]:
# Generate new transformation files based on the excel mapping file. 
# Make sure to have the most updated format for the excel file, check the one used in this notebook for reference.

if ssp_transformation_cw is None:
    logger.warning("ssp_transformation_cw is not defined. Please check your config file.")
else:
    logger.info(f"Using transformation file: {ssp_transformation_cw}")
    cw_file_path = os.path.join(SCENARIO_MAPPING_DIR_PATH, ssp_transformation_cw)
    logger.info(f"Transformation file path: {cw_file_path}")
    excel_yaml_handler = TransformationYamlProcessor(scenario_mapping_excel_path=cw_file_path, yaml_dir_path=TRANSFORMATIONS_DIR_PATH)

In [ ]:
# This creates transformation yaml files for each strategy in the excel file
if ssp_transformation_cw is not None:
    logger.info("Processing YAML files...")
    excel_yaml_handler.process_yaml_files()
else:
    logger.warning("ssp_transformation_cw is not defined. Please check your config file.")

In [ ]:
# Load the transformations per strategy dictionary so we can pass it to the strategy handler
# You can also check here if the transformations in each strategy are correct

if ssp_transformation_cw is not None:
    logger.info("Loading transformations per strategy dictionary...")
    transformation_per_strategy_dict = excel_yaml_handler.get_transformations_per_strategy_dict()
    transformation_per_strategy_dict
    logger.info(f"Loaded transformations for strategies: {transformation_per_strategy_dict.keys()}")
else:
    logger.warning("No transformation handler available. Please check your config file.")

In [ ]:
# You can explore the dictionary to see the transformations per strategy
# transformation_per_strategy_dict

### Creating new strategies
- You can create new strategies from scratch.
- You can also update existing ones.

In [ ]:
# Creating new strategies by updating the strategy_definitions file.

if ssp_transformation_cw is not None:
    # You can edit this to add yours, as many as you want.
    csv_handler = StrategyCSVHandler(csv_file_path=STRATEGIES_DEFINITIONS_FILE_PATH, 
                                     yaml_dir_path=TRANSFORMATIONS_DIR_PATH, 
                                     yaml_mapping_file=STRATEGY_MAPPING_FILE_PATH, 
                                     transformation_per_strategy_dict=transformation_per_strategy_dict)

    csv_handler.add_strategy(strategy_group='PFLO', description='NDC', yaml_file_suffix='NDC')
    csv_handler.add_strategy(strategy_group='PFLO', description='NDC + Energy', yaml_file_suffix='NDC2')
    csv_handler.add_strategy(strategy_group='PFLO', description='Net Zero', yaml_file_suffix='NZ')
else:
    logger.warning("No transformation handler available. Please check your config file.")


### We finished adding new transformation files and strategies so lets load them back

In [ ]:
# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
    TRANSFORMATIONS_DIR_PATH,
    transformers = transformers,
)
tab = transformations.attribute_transformation.table

In [ ]:
#  build the strategies -- will export to path
t0 = time.time()
strategies = trf.Strategies(
    transformations,
    export_path = "transformations",
    prebuild = True,
)

t_elapse = sf.get_time_elapsed(t0)
logger.info(f"Strategies defined at {strategies.transformations.dir_init} initialized in {t_elapse} seconds")

In [ ]:
strategies.attribute_table

In [ ]:
# Set up the strategy codes you wish to run in ssp
strategies_to_run = [0, 6003, 6004, 6005]

##  Build our templates
- let's use the default variable groupings for LHS

In [ ]:
# Building excel templates, make sure to include the strategies ids in the strategies attribute as well as the baseline (0)
df_vargroups = _EXAMPLES("variable_trajectory_group_specification")

strategies.build_strategies_to_templates(
    # df_trajgroup = df_vargroups,
    # include_simplex_group_as_trajgroup = True,
    strategies = strategies_to_run,
)

# Finally, load SISEPUEDE so that we can run it

In [ ]:
import sisepuede as si
# timestamp_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
ssp = si.SISEPUEDE(
    "calibrated",
    db_type = "csv",
    #id_str = f"sisepuede_run_2025-07-28T12:30:52.790396",
    initialize_as_dummy = not(energy_model_flag), # no connection to Julia is initialized if set to True
    regions = [country_name],
    strategies = strategies,
    # try_exogenous_xl_types_in_variable_specification = True,
    attribute_time_period = _ATTRIBUTE_TABLE_TIME_PERIOD
)

In [ ]:
# This runs the model, make sure you edit key_stretegy with the strategy ids you want to execute include baseline (0)
dict_scens = {
    ssp.key_design: [0],
    ssp.key_future: [0],
    ssp.key_strategy: strategies_to_run,
}

ssp.project_scenarios(
    dict_scens,
    save_inputs = True,
    include_electricity_in_energy =energy_model_flag,
)

# Levers Table


In [ ]:
import sisepuede.visualization.tables as svt
tableau_levers_table = svt.LeversImplementationTable(strategies, )
tableau_levers_table_csv = tableau_levers_table.build_table_for_strategies(
    [6003, 6004, 6005]
)

In [ ]:
tableau_levers_table_csv

In [ ]:
# Read input and output files
df_out = ssp.read_output(None)
df_in = ssp.read_input(None)

In [ ]:
df_out[df_out.primary_id==0][[col for col in df_out.columns if col.startswith("prod_ippu_")]].iloc[0]

In [ ]:
df_out[df_out.primary_id==0][[col for col in df_out.columns if col.startswith("prod_ippu_")]]

In [ ]:
demand_ippu =matt.get_variable("Energy Demand by Fuel in SCOE")
demand_ippu.get_from_dataframe(df_out).iloc[35]


In [ ]:
demand_ippu =matt.get_variable("Energy Demand by Fuel in SCOE")
demand_ippu.get_from_dataframe(df_out).iloc[0]

In [ ]:
demand_ippu =matt.get_variable("Energy Demand by Fuel in SCOE")
demand_ippu.get_from_dataframe(df_out).iloc[0]

In [ ]:
frac_demand_electricity_scoe = matt.get_variable("SCOE Heat Energy Demand Scalar")
frac_demand_electricity_scoe.get_from_dataframe(df_in).head()


In [ ]:
frac_demand_electricity_scoe = matt.get_variable("SCOE Initial Per GDP Heat Energy Consumption")
frac_demand_electricity_scoe.get_from_dataframe(df_in).head()

In [ ]:
frac_demand_electricity_scoe = matt.get_variable("SCOE Initial Per GDP Electric Appliances Energy Consumption")
frac_demand_electricity_scoe.get_from_dataframe(df_in).head()

In [ ]:
frac_demand_electricity_scoe = matt.get_variable("SCOE Initial Per Household Electric Appliances Energy Consumption")
frac_demand_electricity_scoe.get_from_dataframe(df_in).head()

In [ ]:
demand_energy_tech = matt.get_variable("Energy Demand by Fuel in Energy Technology")
demand_ccsq = matt.get_variable("Energy Demand by Fuel in CCSQ")
demand_industrial_energy = matt.get_variable("Energy Demand by Fuel in Industrial Energy")
demand_scoe = matt.get_variable("Energy Demand by Fuel in SCOE")
demand_transportation = matt.get_variable("Energy Demand by Fuel in Transportation")
demand_industrial = matt.get_variable("Energy Demand in Industrial Energy")


In [ ]:
demand_scoe.get_from_dataframe(df_out).iloc[0]

In [ ]:
def plot_field_stack(
    df,
    fields,
    dict_format,
    time_col="time_period",
    primary_id=0,
    figsize=(18, 8),
    legend_loc='upper right',
    legend_bbox=(1.1, 1),
    ylabel="MT Emissions CO2e",
    xlabel="Time Period",
    title=None,
):
    """
    Plots a stack plot of the selected fields for a given primary_id.

    Args:
        df (pd.DataFrame): DataFrame containing output data.
        fields (list): List of column names to plot.
        dict_format (dict): Formatting dictionary for colors.
        time_col (str): Name of the time column.
        primary_id (int): Value of primary_id to filter.
        figsize (tuple): Figure size.
        legend_loc (str): Legend location.
        legend_bbox (tuple): Legend bbox_to_anchor.
        ylabel (str): Y-axis label.
        xlabel (str): X-axis label.
        title (str): Plot title.
    """
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if title:
        ax.set_title(title)

    df_plot = df[df[ssp.key_primary].isin([primary_id])]

    fig, ax = spu.plot_stack(
        df_plot,
        fields,
        dict_formatting=dict_format,
        field_x=time_col,
        figtuple=(fig, ax),
    )

    ax.legend(loc=legend_loc, bbox_to_anchor=legend_bbox, title="Fields")
    plt.show()

In [ ]:
## Fix FGTV when having super large spikes (Only use when there are spikes)
primary_id_to_fix = 72072
df_out[df_out.primary_id==primary_id_to_fix][[col for col in df_out.columns if 'fgtv' in col]].plot(figsize=(10,5))

def hampel_clean(
    s: pd.Series,
    window: int = 7,
    n_sigmas: float = 5.0,
    strategy: str = "prev",  # "prev" | "median" | "linear" | "nan"
):
    """
    Detects spikes using a rolling median + MAD (Hampel) and replaces them.

    Returns
    -------
    cleaned : pd.Series
    mask : pd.Series[bool]  # True where a spike was found
    """
    s = s.astype(float).copy()

    # Rolling median
    med = s.rolling(window, center=True, min_periods=max(3, window//2)).median()

    # Rolling MAD (median absolute deviation)
    def _mad(x):
        m = np.median(x)
        return np.median(np.abs(x - m))
    mad = s.rolling(window, center=True, min_periods=max(3, window//2)).apply(_mad, raw=False)

    # Robust z-score using MAD (~= std when multiplied by 1.4826)
    sigma = 1.4826 * mad
    # Fallbacks at edges
    med = med.fillna(s.median())
    sigma = sigma.replace(0, np.nan).fillna(sigma[sigma>0].median() or 1.0)

    z = (s - med).abs() / sigma
    mask = z > n_sigmas  # spike locations (True = spike)

    # Impute
    cleaned = s.copy()
    if strategy == "prev":
        # copy previous *valid* value
        # (for first element or consecutive spikes, fall back to local median)
        for idx in np.where(mask)[0]:
            if idx > 0 and not np.isnan(cleaned.iloc[idx-1]):
                cleaned.iloc[idx] = cleaned.iloc[idx-1]
            else:
                cleaned.iloc[idx] = med.iloc[idx]
    elif strategy == "median":
        cleaned[mask] = med[mask]
    elif strategy == "linear":
        tmp = cleaned.copy()
        tmp[mask] = np.nan
        cleaned = tmp.interpolate(method="linear", limit_direction="both")
    elif strategy == "nan":
        cleaned[mask] = np.nan
    else:
        raise ValueError("Unknown strategy")

    return cleaned, mask

# --- Example on your Series s ---
# s_clean, spike_mask = hampel_clean(s, window=7, n_sigmas=5.0, strategy="prev")

# select the subset
subset = df_out[df_out.primary_id == primary_id_to_fix].copy()

# get only the relevant columns
cols = [c for c in df_out.columns if 'fgtv' in c]

# make a copy to store cleaned results
cleaned_subset = subset.copy()
spike_masks = {}

# apply Hampel cleaning to each relevant column
for c in cols:
    cleaned_subset[c], spike_masks[c] = hampel_clean(
        subset[c],
        window=7,
        n_sigmas=5.0,
        strategy="prev"  # or "median"/"linear"
    )

# if you want to merge back into df_out:
df_out.loc[df_out.primary_id == primary_id_to_fix, cols] = cleaned_subset[cols]
df_out[df_out.primary_id==primary_id_to_fix][[col for col in df_out.columns if 'fgtv' in col]].plot(figsize=(10,5))

In [ ]:
# Define the fields to plot and the formatting dictionary
subsector_emission_fields = matt.get_all_subsector_emission_total_fields()

dict_format = dict(
    (k, {"color": v}) for (k, v) in
    matt.get_subsector_color_map().items()
)

In [ ]:
primary_ids_to_plot = df_out[ssp.key_primary].unique()
primary_ids_to_plot

In [ ]:
# Plot the emissions stack for the primary_id 0 (which is the baseline)
for primary_id in primary_ids_to_plot:

    plot_field_stack(
        df_out,
        subsector_emission_fields,
        dict_format,
        primary_id=primary_id,
        title=f"Emissions Stack Plot for Primary ID {primary_id}"
    )

In [ ]:
df_out[df_out.primary_id==0][[col for col in df_out.columns if col.startswith("prod_ippu_metals_tonne")]]

# Export Wide File (Last Mandatory Step)

In [ ]:
all_primaries = sorted(list(df_out[ssp.key_primary].unique()))

# build if unable to simply read the data frame
if df_in is None:
    df_in = []
     
    for region in ssp.regions:
        for primary in all_primaries: 
            df_in_filt = ssp.generate_scenario_database_from_primary_key(primary)
            df_in.append(df_in_filt.get(region))
    
    df_in = pd.concat(df_in, axis = 0).reset_index(drop = True)




df_export = pd.merge(
    df_out,
    df_in,
    how = "left",
)



# check output directory 
dir_pkg = os.path.join(
    ssp.file_struct.dir_out, 
    f"sisepuede_summary_results_run_{ssp.id_fs_safe}"
)
os.makedirs(dir_pkg) if not os.path.exists(dir_pkg) else None


for tab in ["ATTRIBUTE_STRATEGY"]:
    table_df = ssp.database.db.read_table(tab)
    if table_df is not None:
        table_df.to_csv(
            os.path.join(dir_pkg, f"{tab}.csv"),
            index=None,
            encoding="UTF-8"
        )
    else:
        print(f"Warning: Table {tab} returned None.")


df_primary = (
    ssp
    .odpt_primary
    .get_indexing_dataframe(
        sorted(list(df_out[ssp.key_primary].unique()))
    )
)
    
df_primary.to_csv(
    os.path.join(dir_pkg, f"ATTRIBUTE_PRIMARY.csv"),
    index = None,
    encoding = "UTF-8"
)

df_export.to_csv(
    os.path.join(dir_pkg, f"sisepuede_results_{ssp.id_fs_safe}_WIDE_INPUTS_OUTPUTS.csv"),
    index = None,
    encoding = "UTF-8"
)

In [ ]:
# Getting the directory where the outputs are stored
ssp.file_struct.dir_out

In [ ]:
RUN_ID_OUTPUT_DIR_PATH = os.path.join(
    RUN_OUTPUT_DIR_PATH, 
    f"sisepuede_results_{ssp.id_fs_safe}"
)

os.makedirs(RUN_ID_OUTPUT_DIR_PATH, exist_ok=True)

df_primary.to_csv(
    os.path.join(RUN_ID_OUTPUT_DIR_PATH, "ATTRIBUTE_PRIMARY.csv"),
    index = None,
    encoding = "UTF-8"
)

df_export.to_csv(
    os.path.join(RUN_ID_OUTPUT_DIR_PATH, f"sisepuede_results_{ssp.id_fs_safe}_WIDE_INPUTS_OUTPUTS.csv"),
    index = None,
    encoding = "UTF-8"
)

for tab in ["ATTRIBUTE_STRATEGY"]:
    table_df = ssp.database.db.read_table(tab)
    if table_df is not None:
        table_df.to_csv(
            os.path.join(RUN_ID_OUTPUT_DIR_PATH, f"{tab}.csv"),
            index=None,
            encoding="UTF-8"
        )
    else:
        logger.warning(f"Warning: Table {tab} returned None.")

In [ ]:
tableau_levers_table_csv.to_csv(
            os.path.join(RUN_ID_OUTPUT_DIR_PATH, "tableau_levers_table.csv"),
            index=None,
            encoding="UTF-8"
        )

In [ ]:
RUN_ID_OUTPUT_DIR_PATH

In [ ]:
# gdp_df = df_in[["time_period", "gdp_mmm_usd"]].copy()
# gdp_df["year"] = gdp_df["time_period"] + 2015
# gdp_df = gdp_df.drop(columns=["time_period"])
# gdp_df.head()